# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [93]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
from sklearn.linear_model import Lasso
import statsmodels.api as sm
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.tree import DecisionTreeRegressor


In [94]:
df = pd.read_csv('previsao_de_renda.csv')
data = pd.DataFrame(df)

In [95]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

In [96]:
#1

# Garantir que X e y sejam numéricos
X = X.apply(pd.to_numeric, errors='coerce')
y = pd.to_numeric(y, errors='coerce')

# Variáveis independentes (X) e dependente (y)
X = df.drop('renda', axis=1)
y = df['renda']

# Dividir os dados em conjuntos de treinamento e teste (75% para treinamento, 25% para teste)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
# Verificar o tamanho dos conjuntos
print("Tamanho do conjunto de treinamento:", X_train.shape)
print("Tamanho do conjunto de teste:", X_test.shape)

Tamanho do conjunto de treinamento: (11250, 14)
Tamanho do conjunto de teste: (3750, 14)


In [97]:
#2

# Preencher valores nulos
df['tempo_emprego'].fillna(df['tempo_emprego'].median(), inplace=True)

# Codificação das variáveis categóricas
df = pd.get_dummies(df, drop_first=True)

# Dividir os dados em variáveis independentes (X) e dependente (y)
X = df.drop('renda', axis=1)
y = df['renda']

# Dividir os dados em conjuntos de treinamento e teste (75% treinamento, 25% teste)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Valores de alpha para testar
alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]
r2_scores = []

for alpha in alphas:
    # Criar e treinar o modelo Ridge
    model = Ridge(alpha=alpha)
    model.fit(X_train, y_train)

    # Fazer previsões na base de teste
    y_pred = model.predict(X_test)

    # Avaliar o modelo com o R²
    r2 = r2_score(y_test, y_pred)
    r2_scores.append(r2)

# Exibir os resultados
for alpha, r2 in zip(alphas, r2_scores):
    print(f'Alpha: {alpha}, R² Score: {r2}')

# Encontrar o melhor alpha com base no R²
best_alpha = alphas[np.argmax(r2_scores)]
best_r2 = max(r2_scores)

print(f'Melhor alpha: {best_alpha} com R² Score: {best_r2}')

Alpha: 0, R² Score: 0.26802204227884163
Alpha: 0.001, R² Score: 0.26802204633430404
Alpha: 0.005, R² Score: 0.2680220609254066
Alpha: 0.01, R² Score: 0.2680220755752635
Alpha: 0.05, R² Score: 0.2680220637071694
Alpha: 0.1, R² Score: 0.26802178629490303
Melhor alpha: 0.01 com R² Score: 0.2680220755752635


<ipython-input-97-c74c07a90982>:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['tempo_emprego'].fillna(df['tempo_emprego'].median(), inplace=True)


In [98]:
#3

# Valores de alpha para testar
alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]
r2_scores_lasso = []

for alpha in alphas:
    # Criar e treinar o modelo LASSO
    model = Lasso(alpha=alpha)
    model.fit(X_train, y_train)

    # Fazer previsões na base de teste
    y_pred = model.predict(X_test)

    # Avaliar o modelo com o R²
    r2 = r2_score(y_test, y_pred)
    r2_scores_lasso.append(r2)

# Exibir os resultados
for alpha, r2 in zip(alphas, r2_scores_lasso):
    print(f'Alpha: {alpha}, R² Score: {r2}')


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:1473: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  return fit_method(estimator, *args, **kwargs)
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.855e+11, tolerance: 7.723e+07 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent

Alpha: 0, R² Score: 0.26802029222505375
Alpha: 0.001, R² Score: 0.26802027212645974
Alpha: 0.005, R² Score: 0.2680201579239637
Alpha: 0.01, R² Score: 0.2680199389911513
Alpha: 0.05, R² Score: 0.2680151506637477
Alpha: 0.1, R² Score: 0.2680016177634079


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.854e+11, tolerance: 7.723e+07
  model = cd_fast.enet_coordinate_descent(


In [99]:
# Resultados da regularização Ridge
r2_scores_ridge = [0.26802204227884163, 0.26802204633430404, 0.2680220609254066, 0.2680220755752635, 0.2680220637071694, 0.26802178629490303]

# Comparar os melhores modelos
best_alpha_ridge = alphas[np.argmax(r2_scores_ridge)]
best_r2_ridge = max(r2_scores_ridge)

best_alpha_lasso = alphas[np.argmax(r2_scores_lasso)]
best_r2_lasso = max(r2_scores_lasso)

print(f'Melhor alpha Ridge: {best_alpha_ridge} com R² Score: {best_r2_ridge}')
print(f'Melhor alpha LASSO: {best_alpha_lasso} com R² Score: {best_r2_lasso}')



Melhor alpha Ridge: 0.01 com R² Score: 0.2680220755752635
Melhor alpha LASSO: 0 com R² Score: 0.26802029222505375


In [100]:
#4

def stepwise_selection(X, y,
                       initial_list=[],
                       threshold_in=0.01,
                       threshold_out=0.05,
                       verbose=True):
    """
    Performs stepwise regression to select features.

    Args:
        X (pd.DataFrame): The independent variables.
        y (pd.Series): The dependent variable.
        initial_list (list): An initial list of features to include.
        threshold_in (float): The p-value threshold for adding a feature.
        threshold_out (float): The p-value threshold for removing a feature.
        verbose (bool): Whether to print the steps of the selection.

    Returns:
        list: The list of selected features.
    """

    # Ensure all columns in X are numeric
    # This is crucial for statsmodels compatibility
    X = X.select_dtypes(include=np.number)

    included = list(initial_list)
    while True:
        changed = False
        # Forward step
        excluded = list(set(X.columns) - set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included + [new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed = True
            if verbose:
                print(f'Add  {best_feature:30} with p-value {best_pval:.6}')

        # Backward step
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # Use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max()  # Null if pvalues is empty
        if worst_pval > threshold_out:
            changed = True
            worst_feature = pvalues.idxmax()
            included.remove(worst_feature)
            if verbose:
                print(f'Drop {worst_feature:30} with p-value {worst_pval:.6}')
        if not changed:
            break
    return included

In [101]:
result = stepwise_selection(X_train, y_train)
print('recursos resultantes:')
print(result)

Add  tempo_emprego                  with p-value 0.0
Add  qt_pessoas_residencia          with p-value 1.75831e-12
Add  qtd_filhos                     with p-value 0.000908728
Add  idade                          with p-value 0.000251356
recursos resultantes:
['tempo_emprego', 'qt_pessoas_residencia', 'qtd_filhos', 'idade']


In [102]:
# Criar e treinar o modelo com as variáveis selecionadas
X_train_stepwise = X_train[result]
X_test_stepwise = X_test[result]

model = sm.OLS(y_train, sm.add_constant(X_train_stepwise)).fit()

# Fazer previsões na base de teste
y_pred_stepwise = model.predict(sm.add_constant(X_test_stepwise))

# Avaliar o modelo com o R²
r2_stepwise = r2_score(y_test, y_pred_stepwise)

print(f'R² Score com Stepwise: {r2_stepwise}')


R² Score com Stepwise: 0.1546641272297311


O modelo Ridge com alpha = 0.01 teve o melhor desempenho, com um
𝑅
2
 de 0.268. Ele foi melhor que os modelos LASSO e Stepwise.

In [103]:
#5

from sklearn.linear_model import Ridge
ridge_model = Ridge(alpha=0.01)
ridge_model.fit(X_train, y_train)
ridge_coefs = pd.Series(ridge_model.coef_, index=X_train.columns)

from sklearn.linear_model import Lasso
lasso_model = Lasso(alpha=0)
lasso_model.fit(X_train, y_train)
lasso_coefs = pd.Series(lasso_model.coef_, index=X_train.columns)

import statsmodels.api as sm
stepwise_features = stepwise_selection(X_train, y_train)
X_train_stepwise = X_train[stepwise_features]
X_test_stepwise = X_test[stepwise_features]
stepwise_model = sm.OLS(y_train, sm.add_constant(X_train_stepwise)).fit()
stepwise_coefs = stepwise_model.params




/usr/local/lib/python3.10/dist-packages/sklearn/base.py:1473: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  return fit_method(estimator, *args, **kwargs)
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.855e+11, tolerance: 7.723e+07 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent

Add  tempo_emprego                  with p-value 0.0
Add  qt_pessoas_residencia          with p-value 1.75831e-12
Add  qtd_filhos                     with p-value 0.000908728
Add  idade                          with p-value 0.000251356


In [104]:
comparison_df = pd.DataFrame({
    'Ridge': ridge_coefs,
    'LASSO': lasso_coefs,
    'Stepwise': stepwise_coefs
})
print(comparison_df)


                                     Ridge        LASSO     Stepwise
Unnamed: 0                       -0.204916    -0.177992          NaN
const                                  NaN          NaN   864.470144
data_ref_2015-02-01             225.258953   194.765104          NaN
data_ref_2015-03-01             700.651233   643.288287          NaN
data_ref_2015-04-01            1178.874929  1094.720252          NaN
data_ref_2015-05-01            1049.641831   938.430222          NaN
data_ref_2015-06-01            1608.678167  1470.825072          NaN
data_ref_2015-07-01            1499.102341  1334.004416          NaN
data_ref_2015-08-01            1706.859263  1515.132048          NaN
data_ref_2015-09-01            1928.064140  1709.436748          NaN
data_ref_2015-10-01            1944.179600  1698.684154          NaN
data_ref_2015-11-01            2373.367392  2100.825708          NaN
data_ref_2015-12-01            3054.730604  2755.461903          NaN
data_ref_2016-01-01            278

Avaliação dos Modelos

Modelo Ridge:

Maior
𝑅
2
R
2
 : 0,2680
Usa regularização para evitar overfitting, distribuindo os pesos suavemente entre as variáveis.

Modelo LASSO:

𝑅
2
R
2
  um pouco menor que o Ridge: 0,2680
Também usa regularização, mas zera muitos coeficientes, tornando o modelo mais simples.

Modelo Stepwise:

Menor
𝑅
2
R
2
 : 0,1547
Seleciona variáveis passo a passo, mas é menos robusto que os métodos de regularização.

Conclusão

O modelo Ridge (
𝛼
=
0
,
01
α=0,01) teve o melhor desempenho (
𝑅
2
=
0
,
2680
R
2
 =0,2680), equilibrando bem o uso das variáveis sem zerar muitos coeficientes. Por isso, é o mais indicado para este conjunto de dados.

In [105]:
#6

# Aplicar transformações logarítmicas onde faz sentido
X_train['log_tempo_emprego'] = np.log1p(X_train['tempo_emprego'])
X_test['log_tempo_emprego'] = np.log1p(X_test['tempo_emprego'])

# Criar variáveis interativas e polinomiais
poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)

# Normalizar as variáveis
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_poly)
X_test_scaled = scaler.transform(X_test_poly)

# Treinar e avaliar o modelo Ridge com as novas transformações
ridge_model = Ridge(alpha=0.01)
ridge_model.fit(X_train_scaled, y_train)
y_pred_ridge = ridge_model.predict(X_test_scaled)

# Avaliar o modelo com o R²
r2_transformed_ridge = r2_score(y_test, y_pred_ridge)
print(f'R² Score com Ridge (transformado): {r2_transformed_ridge}')


R² Score com Ridge (transformado): 0.3074348447081805


A transformação e a normalização das variáveis resultaram em uma melhora significativa no
𝑅
2
. O modelo Ridge com as transformações aplicadas conseguiu um
𝑅
2
 de 0.307, o que é um avanço considerável em comparação com os valores anteriores.

Comparação Final dos Resultados
Modelo Ridge (sem transformações):

Melhor
𝑅
2
: 0.268

Modelo LASSO:

Melhor
𝑅
2
: 0.268

Modelo Stepwise:

𝑅
2
: 0.155

Modelo Ridge (com transformações):

Melhor
𝑅
2
: 0.307

Conclusão
Com base nos resultados, o modelo Ridge com transformações apresentou o melhor desempenho, com um
𝑅
2
 de 0.307. Isso demonstra que aplicar transformações logarítmicas e gerar interações entre as variáveis pode ajudar a melhorar a precisão do modelo.

In [106]:
#7

# Criar e treinar a árvore de regressão
tree_model = DecisionTreeRegressor(random_state=42)
tree_model.fit(X_train, y_train)

# Fazer previsões na base de teste
y_pred_tree = tree_model.predict(X_test)

# Avaliar o modelo com o R²
r2_tree = r2_score(y_test, y_pred_tree)
print(f'R² Score com Árvore de Regressão: {r2_tree}')


R² Score com Árvore de Regressão: 0.11752192183511612


Comparando os resultados, observamos que a árvore de regressão não superou os outros modelos, apresentando um
𝑅
2
 de aproximadamente 0.118. Aqui estão os resultados finais:

Comparação dos Resultados
Modelo Ridge (sem transformações):

Melhor
𝑅
2
: 0.268

Modelo LASSO:

Melhor
𝑅
2
: 0.268

Modelo Stepwise:

𝑅
2
: 0.155

Modelo Ridge (com transformações):

Melhor
𝑅
2
: 0.307

Árvore de Regressão:

𝑅
2
: 0.118

Conclusão
O modelo Ridge com transformações permanece como o melhor, com um
𝑅
2
 de 0.307, superando os outros métodos, incluindo a árvore de regressão.